In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
import warnings
import re
from nltk.corpus import stopwords
from transformers import EarlyStoppingCallback
import numpy as np

# Suppress warnings
warnings.filterwarnings('ignore')

# Download NLTK stopwords
nltk.download('stopwords', quiet=True)

# Load data
df = pd.read_csv('/content/train.csv')

# Define stopwords
stop_words = set(stopwords.words('english'))

def clean_text_v2(text):
    if pd.isnull(text):
        return '<empty>'
    text = re.sub(r'[^\w\s!@#$%^&*]', '', text.lower())
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply text cleaning
df['cleaned_comment_text'] = df['comment_text'].apply(clean_text_v2)

# Load tokenizer and model (using ALBERT, a smaller model)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2", use_fast=True)

Label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
results = {}

# Tokenizer function
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=128)  # Reduce max length

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop
for label in Label_columns:
    print(f"\nTraining model for {label}...\n")

    y = df[label]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        df['cleaned_comment_text'], y, test_size=0.2, random_state=42, stratify=y
    )

    # Tokenize data with padding and truncation to max length
    train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=128)
    test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=128)

    class ToxicDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __len__(self):
            return len(self.labels)

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx]).to(device)
            return item

    train_dataset = ToxicDataset(train_encodings, y_train.values)
    test_dataset = ToxicDataset(test_encodings, y_test.values)

    # Load model (using ALBERT)
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    #model = model.to(device)  # Move model to GPU if available

    # Define training arguments with optimizations
    training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    evaluation_strategy="epoch",  # Disable evaluation during training
    save_strategy="epoch",  # Save every epoch
    learning_rate=5e-5,  # Learning rate
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=2,  # Fewer epochs to speed up
    weight_decay=0.01,  # Weight decay
    logging_dir="./logs",  # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,  # Load the best model based on evaluation metrics
    metric_for_best_model="accuracy",  # Metric to track
    fp16=True,  # Enable mixed precision training for GPU acceleration
    gradient_checkpointing=False,  # Reduce memory usage during training
    report_to="none",  # Disable WandB
    dataloader_pin_memory=False,
    )

    # EarlyStoppingCallback with a lower patience
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=1)  # Stop after 1 step of no improvement

    from sklearn.metrics import accuracy_score

    # Define a function to compute evaluation metrics, including accuracy
    def compute_metrics(p):
      predictions, labels = p
      preds = predictions.argmax(axis=-1)  # Get the predicted class labels
      accuracy = accuracy_score(labels, preds)  # Calculate accuracy
      return {'eval_accuracy': accuracy}  # Return the accuracy in the dictionary

    # Trainer setup
    trainer = Trainer(
    model=model.to(device),  # Ensure the model is moved to the correct device (GPU)
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[early_stopping_callback],  # Add early stopping callback
    compute_metrics=compute_metrics,  # Pass the compute_metrics function
    )
    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained(f"./saved_model_{label}")
    tokenizer.save_pretrained(f"./saved_model_{label}")

    # Evaluate the model
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    report = classification_report(y_test, preds, output_dict=True)
    results[label] = report

    print(f"Classification Report for {label}:")
    print(classification_report(y_test, preds))

print("\nSummary of Precision Scores:")
for label, report in results.items():
    print(f"{label}: Precision = {report['1']['precision']:.4f}")

# Save results to CSV
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv('classification_reports_pretrained.csv', index=True)



Training model for toxic...



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.043400,0.152917,0.957512
2,0.134300,0.135832,0.963747


Classification Report for toxic:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     28856
           1       0.87      0.74      0.80      3059

    accuracy                           0.96     31915
   macro avg       0.92      0.86      0.89     31915
weighted avg       0.96      0.96      0.96     31915


Training model for severe_toxic...



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.211900,0.139120,0.963779
2,0.179200,0.127778,0.964092


Classification Report for insult:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     30340
           1       0.65      0.58      0.61      1575

    accuracy                           0.96     31915
   macro avg       0.82      0.78      0.80     31915
weighted avg       0.96      0.96      0.96     31915


Training model for identity_hate...



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.161000,0.053027,0.991195
2,0.000500,0.039191,0.991634


Classification Report for identity_hate:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31634
           1       0.59      0.16      0.25       281

    accuracy                           0.99     31915
   macro avg       0.79      0.58      0.62     31915
weighted avg       0.99      0.99      0.99     31915


Summary of Precision Scores:
toxic: Precision = 0.8663
severe_toxic: Precision = 0.0000
obscene: Precision = 0.8388
threat: Precision = 0.0000
insult: Precision = 0.6535
identity_hate: Precision = 0.5946
